In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sizlingdhairya1/leaf-disease")

print("Path to dataset files:", path)

100%|██████████| 3.75M/3.75M [00:00<00:00, 98.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sizlingdhairya1/leaf-disease/versions/1


In [2]:
! ls $path/Datasets

Bacteria  Fungi  Nematodes  Normal  Virus


In [29]:
BASE_DIR = f'{path}/Datasets'

## Create more images by rotating and reflecting

In [31]:
from PIL import Image, ImageOps

def create_img(image_path, output_path):
    try:
        img = Image.open(image_path)
        rotated_img = img.rotate(180, expand=True)
        rotated_img.save(output_path)
        mirrored_img = ImageOps.mirror(image_path)
        mirrored_img.save(output_path)
    except FileNotFoundError:
        print(f'Error: Image file not found at {image_path}')
    except Exception as e:
        print(f'An error occurred: {e}')

def new_img_name(old_name, prefix):
    path


## Add all paths and classes to list

In [25]:
import os
from collections import defaultdict

keys = os.listdir(BASE_DIR)

data = defaultdict(list)
for cls in keys:
    img_path = os.listdir(BASE_DIR+'/'+cls)
    data['img_path'].extend(img_path)
    data['class'].extend([cls]*len(img_path))


In [26]:
import pandas as pd
import numpy as np

In [27]:
df = pd.DataFrame(data=data)
df.head()

,img_path,class
0,0a1319ef-0cbb-4fcc-8126-9c18813f3bac___JR_HL 4...,Normal
1,0adc1c5b-8958-47c0-a152-f28078c214f1___RS_HL 7...,Normal
2,0de7baec-4dc0-4778-b3be-34820288f09f___JR_HL 3...,Normal
3,0a68b3a9-e38f-45b5-8664-9c60e557a41d___JR_HL 9...,Normal
4,0ce4ee15-d59b-4212-ab2a-8b6a07f3634f___JR_HL 9...,Normal
